# core

> Code documentation

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
import pandas as pd

class SACCalculator:
    "A simple calculator class"
    def __init__(self, 
                 valor_financiado: float,     # Valor financiado
                 prazo: int,                  # Prazo em meses 
                 taxa_anual: float            # Taxa de juros anual
    ):
        self.valor_financiado = valor_financiado
        self.prazo = prazo
        self.taxa_juros = (1 + taxa_anual/100)**(1/12) - 1
        self.amortizacao = self.valor_financiado / self.prazo

    def tabela(self):
        """
        Gera a tabela SAC
        """
        parcelas = np.array([p for p in range(1, self.prazo+1)])
        juros = np.array([self.taxa_juros * (self.valor_financiado - p * self.amortizacao) for p in range(self.prazo)])
        amortizacao = np.array([self.amortizacao for p in range(1, self.prazo+1)])
        valor_da_parcela = juros + amortizacao
        saldo_devedor = np.array([self.valor_financiado - p * self.amortizacao for p in range(1, self.prazo+1)])
        
        return (
            pd.DataFrame({
                'Parcela': parcelas,
                'Amortização': amortizacao,
                'Juros': juros,
                'Valor da Parcela': valor_da_parcela,
                'Saldo Devedor': saldo_devedor
            }).round(decimals=2)   
        )
    
    def amortizacao_extra_mensal(self, valor_amortizado_extra_mensal: float):
        
        amortizacao = self.valor_financiado / self.prazo
        taxa_mensal = self.taxa_juros
        
        # Listas para armazenar os dados da tabela
        meses_extra = []
        saldos_devedores_extra = []
        amortizacoes_extra = []
        juros_extra = []
        prestacoes_extra = []
        amortizacoes_extras = []

        # Saldo devedor inicial é o valor financiado
        saldo_devedor = self.valor_financiado

        # Cálculo da tabela SAC com amortização extra
        mes = 0
        while saldo_devedor >= 0:
            mes += 1
            meses_extra.append(mes)
            saldos_devedores_extra.append(saldo_devedor)
            amortizacoes_extra.append(amortizacao)
            juro = saldo_devedor * taxa_mensal
            juros_extra.append(juro)
            prestacao = amortizacao + juro
            prestacoes_extra.append(prestacao)
            
            # Abate a amortização extra do saldo devedor
            saldo_devedor -= (amortizacao + min(valor_amortizado_extra_mensal, saldo_devedor))
            amortizacoes_extras.append(min(valor_amortizado_extra_mensal, saldo_devedor))

        # Criação da tabela SAC com amortização extra
        tabela_sac_extra = pd.DataFrame({
            'Parcela': meses_extra,
            'Saldo Devedor Inicial': saldos_devedores_extra,
            'Amortização': amortizacoes_extra,
            'Amortização Extra': amortizacoes_extras,
            'Juros': juros_extra,
            'Valor da Parcela': prestacoes_extra,
            'Saldo Devedor Final': np.roll(saldos_devedores_extra, -1)
        })

        # Exclusão da última linha, que contém dados incorretos devido ao deslocamento
        tabela_sac_amortizada = tabela_sac_extra[:-1]

        return tabela_sac_amortizada

In [ ]:
sac = SACCalculator(205385.45, 420, 9.99)

In [ ]:
tabela = sac.tabela()
tabela.head()

,Parcela,Amortização,Juros,Valor da Parcela,Saldo Devedor
0,1,489.01,1636.20,2125.22,204896.44
1,2,489.01,1632.31,2121.32,204407.42
2,3,489.01,1628.41,2117.43,203918.41
3,4,489.01,1624.52,2113.53,203429.40
4,5,489.01,1620.62,2109.63,202940.39


In [ ]:
tabela_amortizacao_extra = sac.amortizacao_extra_mensal(valor_amortizado_extra_mensal=5000)
tabela_amortizacao_extra.head()

,Parcela,Saldo Devedor Inicial,Amortização,Amortização Extra,Juros,Valor da Parcela,Saldo Devedor Final
0,1,205385.450000,489.012976,5000.0,1636.203997,2125.216973,199896.437024
1,2,199896.437024,489.012976,5000.0,1592.475754,2081.488730,194407.424048
2,3,194407.424048,489.012976,5000.0,1548.747510,2037.760486,188918.411071
3,4,188918.411071,489.012976,5000.0,1505.019267,1994.032243,183429.398095
4,5,183429.398095,489.012976,5000.0,1461.291023,1950.303999,177940.385119


In [ ]:
len(tabela.index), len(tabela_amortizacao_extra)

(420, 37)

In [ ]:
tabela['Valor da Parcela'].sum(), tabela_amortizacao_extra['Saldo Devedor Inicial'].loc[0] + tabela_amortizacao_extra['Juros'].sum()

(549806.39, 236801.98771555116)

In [ ]:
tabela['Juros'].sum(), tabela_amortizacao_extra['Juros'].sum()

(344420.94, 31416.53771555116)

In [ ]:
assert SACCalculator(200000, 360, 0.01).tabela().shape[0] == 360

In [ ]:
assert SACCalculator(200000, 12, 0.01).tabela().shape[0] == 12

In [ ]:
assert SACCalculator(200000, 24, 0.01).tabela().shape[0] == 24

In [ ]:
assert SACCalculator(200000, 5, 0.01).tabela().shape[0] == 5

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()